# Energy Consumption
In this notebook we will explore energy consumption data, to see if it is useful for our data challenge.

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)

print('Pandas version:', pd.__version__) # 2.2.3
print('Numpy version:', np.__version__) # 1.26.4
print('Seaborn version:', sns.__version__) # 0.13.2

Pandas version: 2.2.3
Numpy version: 1.26.4
Seaborn version: 0.13.2


# Load the dataset

In [6]:
df = pd.read_csv('../data/ConsumptionData/cleaned_consumption_data.csv')
df.head()

,Date,Bruto productie (mln kWh),Distributieverliezen (mln kWh),Eigen verbruik bij elektr.productie (mln kWh),Invoer|België (mln kWh),Invoer|Denemarken (mln kWh),Invoer|Duitsland (mln kWh),"Invoer|Invoer, totaal (mln kWh)",Invoer|Noorwegen (mln kWh),Invoer|Verenigd Koninkrijk (mln kWh),Netto productie|Brandstoffen|Aardgas (mln kWh),Netto productie|Brandstoffen|Biomassa (mln kWh),"Netto productie|Brandstoffen|Brandstoffen, totaal (mln kWh)",Netto productie|Brandstoffen|Kolen (mln kWh),Netto productie|Brandstoffen|Olieproducten (mln kWh),Netto productie|Brandstoffen|Overige brandstoffen (niet-hernieuwbaar) (mln kWh),Netto productie|Kernenergie (mln kWh),"Netto productie|Netto productie, totaal (mln kWh)",Netto productie|Overige bronnen (mln kWh),Netto productie|Waterkracht (mln kWh),Netto productie|Windenergie|Windenergie op land (mln kWh),Netto productie|Windenergie|Windenergie op zee (mln kWh),"Netto productie|Windenergie|Windenergie, totaal (mln kWh)",Netto productie|Zonnestroom (mln kWh),Netto verbruik (berekend) (mln kWh),Uitvoer|België (mln kWh),Uitvoer|Denemarken (mln kWh),Uitvoer|Duitsland (mln kWh),Uitvoer|Noorwegen (mln kWh),"Uitvoer|Uitvoer, totaal (mln kWh)",Uitvoer|Verenigd Koninkrijk (mln kWh)
0,2021-01-01,11647.0,447.0,345.0,184.0,153.0,1075.0,1542.0,130.0,0.0,6113.0,936.0,8897.0,1596.0,110.0,142.0,358.0,11303.0,34.0,7.0,943.0,868.0,1811.0,196.0,10731.0,1146.0,97.0,368.0,56.0,1667.0,0.0
1,2021-02-01,10907.0,401.0,296.0,460.0,180.0,730.0,1378.0,0.0,7.0,5882.0,819.0,7948.0,1037.0,102.0,109.0,321.0,10610.0,38.0,4.0,942.0,913.0,1854.0,444.0,9611.0,600.0,152.0,845.0,0.0,1977.0,380.0
2,2021-03-01,11626.0,412.0,322.0,393.0,242.0,605.0,1269.0,30.0,0.0,6091.0,875.0,8412.0,1212.0,99.0,135.0,360.0,11303.0,42.0,12.0,911.0,731.0,1642.0,835.0,9961.0,595.0,104.0,1296.0,0.0,2199.0,204.0
3,2021-04-01,9784.0,378.0,288.0,253.0,279.0,473.0,1384.0,380.0,0.0,4557.0,789.0,6410.0,799.0,130.0,135.0,333.0,9496.0,42.0,8.0,743.0,628.0,1371.0,1332.0,8791.0,666.0,73.0,924.0,47.0,1711.0,0.0
4,2021-05-01,9241.0,379.0,274.0,833.0,252.0,414.0,1868.0,370.0,0.0,4234.0,725.0,5918.0,714.0,104.0,142.0,210.0,8967.0,41.0,9.0,786.0,519.0,1305.0,1484.0,8963.0,435.0,73.0,913.0,71.0,1492.0,0.0


# Data Analysis

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 31 columns):
 #   Column                                                                           Non-Null Count  Dtype  
---  ------                                                                           --------------  -----  
 0   Date                                                                             36 non-null     object 
 1   Bruto productie (mln kWh)                                                        36 non-null     float64
 2   Distributieverliezen (mln kWh)                                                   36 non-null     float64
 3   Eigen verbruik bij elektr.productie (mln kWh)                                    36 non-null     float64
 4   Invoer|België (mln kWh)                                                          36 non-null     float64
 5   Invoer|Denemarken (mln kWh)                                                      36 non-null     float64
 6   Invoer|Duits